# EP01: 대규모 쿠버네티스 클러스터 만들기

Q: 이번에 살펴볼 내용은 어떤 내용인가요?

A: 이번 에피소드에서는 수십대의 쿠버네티스 노드를 필요에 따라 빠르게 확장하거나 축소할 수 있는 쿠버네티스 클러스터를 AKS 엔진으로 직접 만드는 내용을 살펴보려고 합니다.

Q: 대규모로 쿠버네티스 클라우드를 운영하기 위해서는 고려해야 할 내용들이 많이 있을 것 같은데, 어떤게 있을까요?

A: 기본적으로 쿠버네티스 클러스터는 대규모로 운영하더라도 잘 작동할 수 있도록 설계되어있습니다. 그래서 규모가 크든 작든 사용 방법에서는 차이가 없습니다.

다만 쿠버네티스가 실행되는 기반은 애저이고, 애저는 여러 사람들이 공용으로 사용하는 컴퓨팅 자원을 여러분에게 제공합니다.

그래서 세 가지 사항을 고려하시는 것이 좋습니다.

A: 첫 째는 노드 풀의 크기를 작은 단위로 여러 개를 준비하는 것이 좋습니다. 예를 들어, 노드 풀 한 개에는 5개에서 20개 정도의 노드를 만들 수 있도록 설정하고, 이런 노드 풀을 50개, 100개 이상 미리 준비하여 필요할 때 바로 노드를 만들고, 필요 없을 때 해제할 수 있도록 만들어 두는 것입니다.

A: 둘 째는 뒤에서 다시 살펴보겠지만, 재시도를 하는 간격 사이를 지수승 간격만큼 벌리는 것입니다. 애저로부터 노드를 할당받을 수 없는 상황이 오면, 보통은 일시적으로 요청량이 크게 늘어난 경우가 대체로 많습니다. 이럴 때에는 무한정 재시도를 하는 것보다 재시도 횟수 간의 간격을 점점 크게 잡는 것이 성공 확률을 높이고, 클라우드 전반의 안정성을 높이는데도 도움이 됩니다.

마지막으로는 할당량 제한을 항상 모니터링하고, 수요에 따라 미리 할당량 증가 신청을 접수하는 것입니다. 이 부분에 대해서는 나중에 다시 말씀드리겠습니다.

## 애저 로그인 상태 확인

우선 애저 커맨드 라인 도구의 로그인 상태를 먼저 확인합니다. 로그인 상태가 아니면 다른 터미널에서 `az login` 명령을 실행합니다. 설치되어있지 않다면 OS 버전 별로 애저 커맨드 라인 도구를 설치하고 준비합니다.

In [1]:
az account list

The below script needs to be able to find the current output cell; this is an easy method to get it.

Name               CloudName    SubscriptionId                        State    IsDefault
-----------------  -----------  ------------------------------------  -------  -----------
rkttu@outlook.com  AzureCloud   e6535fda-2c47-411d-9e51-ae11ef7bde27  Enabled  True
DES File Shares    AzureCloud   173130ce-0596-48f7-a0e4-10c465471285  Enabled  False


## AKS 엔진 설치 상태 확인

아래 명령어를 실행하여 `aks-engine`이 설치되었는지 확인합니다. 설치되어있지 않다면 OS 버전 별로 각각 설치 가이드를 따라 설치를 진행합니다.

In [2]:
aks-engine version

Version: v0.56.0
GitCommit: 666073d49
GitTreeState: clean


## 서비스 주체 생성

쿠버네티스 클러스터를 만든 후에는 클러스터 수준에서 스스로 애저 리소스들을 제어할 수 있어야 하므로, 서비스 주체 계정을 만들어 계속 해당 계정을 사용하도록 설정합니다.

In [3]:
$SubscriptionData = (az account list -o json) | ConvertFrom-Json -Depth 16
$SubscriptionId = $SubscriptionData[0].id
$SvcPrincipal = (az ad sp create-for-rbac `
  -n="AzurePlaylist" --role="Contributor" `
  --scopes="/subscriptions/$SubscriptionId" `
  -o json) | ConvertFrom-Json -Depth 16

$PrincipalId = $SvcPrincipal.AppId
$PrincipalPwd = $SvcPrincipal.Password

"Your App ID is $PrincipalId"

Changing "AzurePlaylist" to a valid URI of "http://AzurePlaylist", which is the required format used for service principal names
Found an existing application instance of "5f6ed45a-660e-47f5-9c08-6e5515f17640". We will patch it
Creating a role assignment under the scope of "/subscriptions/e6535fda-2c47-411d-9e51-ae11ef7bde27"
  Role assignment already exists.

Your App ID is 5f6ed45a-660e-47f5-9c08-6e5515f17640


## SSH 키 준비하기

아래 명령어를 실행하여 SSH 키의 공개 키를 준비합니다. 설치되어있지 않다면 SSH 키 쌍을 만듭니다.

In [4]:
$PublicKeyValue = Get-Content -Path "$HOME\.ssh\id_rsa.pub"

## API 모델 준비하기

이제 API 모델을 준비할 차례입니다. AKS 엔진은 코드 기반 인프라 관리 방식을 구현하기 위하여 JSON API 모델을 작성하면, 여기에 맞추어 ARM 템플릿을 만들어줍니다. 이것을 애저 명령줄 도구를 통해 배포하면 손쉽게 쿠버네티스 클러스터를 생성할 수 있습니다.

In [5]:
$ApiModel = @"
{
  'apiVersion': 'vlabs',
  'properties': {
    'orchestratorProfile': {
      'orchestratorType': 'Kubernetes',
      'orchestratorRelease': '1.16',
      'kubernetesConfig': {
        'cloudProviderBackoff': true,
        'cloudProviderBackoffRetries': 6,
        'cloudProviderBackoffJitter': 1,
        'cloudProviderBackoffDuration': 6,
        'cloudProviderBackoffExponent': 1.5,
        'cloudProviderRateLimit': true,
        'cloudProviderRateLimitQPS': 3,
        'cloudProviderRateLimitBucket': 10,
        'kubeletConfig': {
          '--node-status-update-frequency': '1m'
        },
        'controllerManagerConfig': {
          '--node-monitor-grace-period': '5m',
          '--pod-eviction-timeout': '1m',
          '--route-reconciliation-period': '1m'
        }
      }
    },
    'servicePrincipalProfile': {
      'clientId': '',
      'secret': ''
    },
    'masterProfile': {
      'count': 1,
      'vmSize': 'Standard_D5_v2'
    },
    'agentPoolProfiles': [
      {
        'name': 'pool1',
        'count': 5,
        'vmSize': 'Standard_D3_v2',
        'availabilityProfile': 'AvailabilitySet'
      },
      {
        'name': 'pool2',
        'count': 5,
        'vmSize': 'Standard_D3_v2',
        'availabilityProfile': 'AvailabilitySet'
      },
      {
        'name': 'pool3',
        'count': 5,
        'vmSize': 'Standard_D3_v2',
        'availabilityProfile': 'AvailabilitySet'
      },
      {
        'name': 'pool4',
        'count': 5,
        'vmSize': 'Standard_D3_v2',
        'availabilityProfile': 'AvailabilitySet'
      },
      {
        'name': 'pool5',
        'count': 5,
        'vmSize': 'Standard_D3_v2',
        'availabilityProfile': 'AvailabilitySet'
      }
    ],
    'linuxProfile': {
      'adminUsername': 'azureuser',
      'ssh': {
        'publicKeys': [
          {
            'keyData': ''
          }
        ]
      }
    }
  }
}
"@ | ConvertFrom-Json -Depth 16

## API 모델 내용 변경하기

나중에 리눅스 마스터 노드에 접속할 수 있도록 현재 컴퓨터에서 사용하는 SSH 공개 키를 배포할 때 같이 등록합니다. 그리고 서비스 주체를 가리키는 아이디 값과 고유 비밀 키 값을 API 모델에 같이 지정합니다.

In [6]:
$ApiModel.properties.linuxProfile.ssh.publicKeys[0].keyData = "$PublicKeyValue"
$ApiModel.properties.servicePrincipalProfile.clientId = "$PrincipalId"
$ApiModel.properties.servicePrincipalProfile.secret = "$PrincipalPwd"

In [7]:
$ApiModel | ConvertTo-Json -Depth 16 | Out-File -Encoding utf8 -Path 'ep01.json' -Force

dir 'ep01*'



    Directory: /Users/rkttu/Projects/azure-playlist-aks-engine

Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
-----      2020. 12. 5. 오전 12:28          17276 EP01.ipynb
-----      2020. 12. 5. 오전 12:28           2395 ep01.json



## API 모델을 ARM 템플릿으로 변환하고 배포하기

다음의 명령어를 실행하여 API 모델을 ARM 템플릿으로 변환하고 곧바로 애저에 배포하도록 하겠습니다. 이 명령어는 최초에 한 번만 실행하고, 그 이후에는 `ep01_output` 디렉터리 안에 있는 파일들을 이용하여 클러스터를 관리하도록 합니다.

In [11]:
$DnsPrefix = 'rkttuep01'
$ResourceGroup = 'rkttuep01'
$Location = 'westus2'

aks-engine deploy --dns-prefix "$DnsPrefix" `
    --resource-group "$ResourceGroup" `
    --location "$Location" `
    --api-model "ep01.json" `
    --auto-suffix `
    --output-directory "ep01_output" `
    --auth-method cli

INFO[0000] No subscription provided, using selected subscription from azure CLI: e6535fda-2c47-411d-9e51-ae11ef7bde27 
WARN[0002] apimodel: missing masterProfile.dnsPrefix will use "rkttuep01" 
WARN[0004] containerd will be upgraded to version 1.3.7 
INFO[0007] Starting ARM Deployment rkttuep01-1713952306 in resource group rkttuep01. This will take some time... 
INFO[0304] Finished ARM Deployment (rkttuep01-1713952306). Succeeded 


## kubectl 실행해보기

클러스터가 잘 만들어졌는지 확인해보기 위하여 다음과 같이 `KUBECONFIG` 환경 변수를 설정하여 클러스터에 접근하고, 노드와 구성 상태를 확인합니다.

In [12]:
$env:KUBECONFIG="ep01_output/kubeconfig/kubeconfig.$Location.json"
kubectl get nodes -o wide

NAME                    STATUS     ROLES    AGE   VERSION    INTERNAL-IP    EXTERNAL-IP   OS-IMAGE             KERNEL-VERSION     CONTAINER-RUNTIME
k8s-master-82468032-0   Ready      master   75s   v1.16.15   10.255.255.5   <none>        Ubuntu 18.04.5 LTS   5.4.0-1025-azure   docker://19.3.12
k8s-pool1-82468032-0    Ready      agent    75s   v1.16.15   10.240.2.111   <none>        Ubuntu 18.04.5 LTS   5.4.0-1025-azure   docker://19.3.12
k8s-pool1-82468032-1    Ready      agent    75s   v1.16.15   10.240.0.221   <none>        Ubuntu 18.04.5 LTS   5.4.0-1025-azure   docker://19.3.12
k8s-pool1-82468032-2    Ready      agent    74s   v1.16.15   10.240.2.18    <none>        Ubuntu 18.04.5 LTS   5.4.0-1025-azure   docker://19.3.12
k8s-pool1-82468032-3    Ready      agent    75s   v1.16.15   10.240.1.57    <none>        Ubuntu 18.04.5 LTS   5.4.0-1025-azure   docker://19.3.12
k8s-pool1-82468032-4    Ready      agent    75s   v1.16.15   10.240.1.88    <none>        Ubuntu 18.04.5 LTS   5.4.0-

## 리소스 그룹 정리

테스트를 위해 생성한 리소스 그룹을 제거하고 모든 리소스를 일괄 정리합니다.

In [13]:
az group delete --name $ResourceGroup --yes --no-wait

Q: 만약에 이보다 더 큰 규모의 클러스터를 만들고 싶다면 알아야 할 것이 무엇이 있을까요?

A: 지역별로 할당되는 인스턴스의 숫자는 다른 퍼블릭 클라우드 공급자들처럼 제한되어있습니다. 만약 원하는 댓수만큼 인스턴스를 늘리고 싶다면, 애저 고객 지원 센터를 통해 원하는 대수를 직접 요청할 수 있습니다.

사용 가능한 할당량을 보는 방법은 해당되는 구독 - 설정 - 사용량 및 할당량 메뉴에서 공급자를 Microsoft.Compute로 설정하면 확인할 수 있습니다.

A: 할당량 증가에 관한 자세한 내용은 [http://bit.ly/rkttu-increase-azure-quota](http://bit.ly/rkttu-increase-azure-quota) 를 참고하세요. 간단한 할당량 증가 신청은 언제든 바로 신청할 수 있고, 빠른 시간 안에 할당량 조절 승인을 받을 수 있습니다.

클로징: 이 동영상을 보고 애저를 써봐야겠다고 생각하셨나요? 애저 계정은 누구나 무료로 만들 수 있습니다. 지금 바로 시작해보세요. (가능하다면 할인/무료 쿠폰을 포함한 가입 링크 제공) 다음 에피소드에서는 클라우드 셸을 사용할 수 있는 다양한 방법을 알아보겠습니다.